In [1]:
import numpy as np
import random

In [2]:
def square_matrix(num_row, num_col):
    if num_row == num_col:
        return True
    else:
        return False

In [3]:
'''
Primero definimos nuestras clases de excepciones, las cuales
nos arrojarán los errores que se espera de capturen dado si 
la matríz ingresada no es cuadrada o invertible.
'''

class SquareMatrixException(Exception): # clase para matríz A cuadrada
    def __init__(self, n, message="Estás ingresando una Matríz que no es cuadrada, por lo tanto no hay factorización"):
        self.n = n
        self.message = message
        super().__init__(self.message)
    
class InvertMatrixException(Exception): # clase para matríz A inversa
    def __init__(self, n, message="Estás ingresando una Matríz que no es invertible, por lo tanto no hay factorización"):
        self.n = n
        self.message = message
        super().__init__(self.message)

## Jacobi

In [4]:
'''
Función que resuelve un sistema de ecuaciones por método Jacobi, en donde se dan los siguientes parámetros:
A = Matríz asociada al sistema
b = Matríz columna asociada a la solución del sistema
e = Número de tolerancia de convergencia, si no se declara, por default es 0.00009
n = Número de iteraciones en el algoritmo, si no se declara, por default es 1000

El formato de las Matríces introducidas son [[a1,...,an],...], ya que dentro de la función se espera tranformar a 
np.matrix
'''

def Jacobi(A, b, e = 0.00009, n = 1000):
    upper_limit = 10000000
    A = np.matrix(A)
    A = A.astype(float)
    b = np.matrix(b)
    b = b.astype(float)
    x_length, y_length = A.shape   
    
    if x_length != y_length:
        raise SquareMatrixException(A)
        
    if np.linalg.det(A) == 0:
        raise InvertMatrixException(A)
        
    # Declaramos las matrices D y E:    
    D = np.zeros((x_length,y_length))
    D_inverse = np.zeros((x_length,y_length))
    for i in range(x_length):
        D[i,i] += A[i,i]
    
    D_inverse = np.linalg.inv(D)    
    E = A - D
    x = np.zeros((x_length,1), dtype=float)

    count = 0
    
    while count <= n:
        try:
            x_k = x.copy()
            x = D_inverse @ b - D_inverse@E@x
            difference = x - x_k
            norma = np.linalg.norm(difference)

        # Comprobación de que la distancia sea menor a e
            if norma < e:
                return x
            if norma > upper_limit:
                return 'No converge la matríz'
            count += 1
        except Warning:
            return 'No converge la matríz'
    else:
        return x

### Test de eficacia del algoritmo

In [5]:
def test_solution(A,b):
    matrix_norm = Jacobi(A,b)
    Ax_b = A@matrix_norm - b
    return np.linalg.norm(Ax_b)

In [6]:
def random_matrix():
    A_matrix_random = []
    b_matrix_random = []
    lenght = random.randint(1,10)

    for i in range(lenght):
        rows_A = []
        for j in range(lenght):
            n = random.randint(-70,70)
            rows_A.append(n)
        A_matrix_random.append(rows_A)
        b_matrix_random.append([random.randint(-70,70)])
        
    return A_matrix_random, b_matrix_random

In [7]:
def test_final(n):
    E = 0.009
    exitos = 0
    for i in range(n):
        try: 
            A, b = random_matrix()
            norma_test = test_solution(A, b)
            if norma_test < E:
#                print(f'norma entre la solución de Jacobi y solución real = {norma_test} en la iteración {i}')
#                print(f'Solución Jacobi = {Jacobi(A,b)}, Ax = {A@Jacobi(A,b)} aprox b = {b}')
                exitos += 1
                continue
        except: 
            '''Al parecer alguno de estos sistemas no se pudo solucionar, 
            ya sea por singularidad'''
    else:
        return f'Nuestro programa funciona bien con {n} iteraciones y un total de {exitos} éxitos.'

In [8]:
test_final(1000)

'Nuestro programa funciona bien con 1000 iteraciones y un total de 174 éxitos.'

Como lo pudimos ver anteriormente, este algoritmo no simepre funciona, tiene sus ventajas como el ser iterativo, sin embargo podemos decir que no es el más certero a la hora de dar soluciones, ya que en algunos casos no las puede dar. De hecho si repetimos los test, aproximadamente de n iteraciones, tenemos que acerta en uno de diez, es decir el $\frac{n}{10}$ aprox de veces nos da una solución.

In [9]:
for i in range(10,10000,1000):
    print(test_final(i))

Nuestro programa funciona bien con 10 iteraciones y un total de 4 éxitos.
Nuestro programa funciona bien con 1010 iteraciones y un total de 173 éxitos.
Nuestro programa funciona bien con 2010 iteraciones y un total de 341 éxitos.
Nuestro programa funciona bien con 3010 iteraciones y un total de 488 éxitos.
Nuestro programa funciona bien con 4010 iteraciones y un total de 660 éxitos.
Nuestro programa funciona bien con 5010 iteraciones y un total de 826 éxitos.
Nuestro programa funciona bien con 6010 iteraciones y un total de 1016 éxitos.
Nuestro programa funciona bien con 7010 iteraciones y un total de 1137 éxitos.
Nuestro programa funciona bien con 8010 iteraciones y un total de 1337 éxitos.
Nuestro programa funciona bien con 9010 iteraciones y un total de 1480 éxitos.


## Gauss - Seidel

Recordemos que para "garantizar" convergencia en este método debe cumplirse lo siguiente:

Sea $A$ una matríz, entonces decimos que hay convergencia en el método sí $$|A_{ii}| \geq \sum_{j=1}^n |A_{ij}| \text{ para } i\neq j$$ De esta forma hagamos la clase de excepción

In [10]:
'''
Definimos nuestras clases de excepciones, las cuales nos arrojarán los errores que se espera de capturen dado si 
la matríz ingresada no converge.
'''

class ConvergenceException(Exception): # clase para matríz A cuadrada
    def __init__(self, n, message="Estás ingresando una Matríz que no converge, por lo tanto no sirve el método aquí."):
        self.n = n
        self.message = message
        super().__init__(self.message)

In [11]:
'''
Función que resuelve un sistema de ecuaciones por método Gauss-Seidel, en donde se dan los siguientes parámetros:
A = Matríz asociada al sistema
b = Matríz columna asociada a la solución del sistema
e = Número de tolerancia de convergencia, si no se declara, por default es 0.00009
n = Número de iteraciones en el algoritmo, si no se declara, por default es 1000

El formato de las Matríces introducidas son [[a1,...,an],...], ya que dentro de la función se espera tranformar a 
np.matrix
'''

def GaussSeidel(A, b, e = 0.00009, n = 1000):
    upper_limit = 1000000 # Esto para manejar convergencia, así nos "aseguramos" que no van a diverger a un número muy grande
    A = np.matrix(A)
    A = A.astype(float)
    b = np.matrix(b)
    b = b.astype(float)
    x_length, y_length = A.shape   
    
    if x_length != y_length:
        raise SquareMatrixException(A)
        
    if np.linalg.det(A) == 0:
        raise InvertMatrixException(A)
        
    # Declaramos las matrices D y E:    
    D = np.zeros((x_length,y_length))
    D_inverse = np.zeros((x_length,y_length))
    for i in range(x_length):
        D[i,i] += A[i,i]
    
    D_inverse = np.linalg.inv(D)    
    E = A - D
    x = np.zeros((x_length,1), dtype=float)
    
    for i in range(x_length):
        if abs(D[i,i]) < np.sum(E[i][:]):
            raise ConvergenceException(A)
        else:
            continue

    count = 0
    
    while count <= n:
        try:
            if count == 0:
                x_k = x.copy()
                x = D_inverse @ b - D_inverse@E@x
                difference = x - x_k
                norma = np.linalg.norm(difference)
            else:
                x_copy = x.copy()
                for i in range(x_length):
                    xk_copy = D_inverse @ b - D_inverse@E@x_copy
                    x[i,0] = xk_copy[i,0]
                    if norma > upper_limit:
                        return 'No converge la matríz'
            if norma < e:
                    return x
            count += 1    
            
        except Warning:
            return 'No converge la matríz Warning'

    else:
        return x

In [12]:
def test_solutionGauss(A,b):
    matrix_norm = GaussSeidel(A,b)
    Ax_b = A@matrix_norm - b
    return np.linalg.norm(Ax_b)

In [13]:
def test_finalGauss(n):
    E = 0.009
    exitos = 0
    for i in range(n):
        try: 
            A, b = random_matrix()
            norma_test = test_solutionGauss(A, b)
            if norma_test < E:
                exitos += 1
                continue
        except: 
            '''Al parecer alguno de estos sistemas no se pudo solucionar, 
            ya sea por singularidad'''
    else:
        return f'Nuestro programa funciona bien con {n} iteraciones y un total de {exitos} éxitos.'

In [14]:
test_finalGauss(100)

/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/829820542.py:54: RuntimeWarning: overflow encountered in matmul
  xk_copy = D_inverse @ b - D_inverse@E@x_copy
/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/829820542.py:54: RuntimeWarning: invalid value encountered in matmul
  xk_copy = D_inverse @ b - D_inverse@E@x_copy
/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/2454541577.py:3: RuntimeWarning: overflow encountered in matmul
  Ax_b = A@matrix_norm - b
/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/2454541577.py:3: RuntimeWarning: invalid value encountered in matmul
  Ax_b = A@matrix_norm - b


'Nuestro programa funciona bien con 100 iteraciones y un total de 10 éxitos.'

Como lo pudimos ver anteriormente, este algoritmo no simepre funciona, tiene sus ventajas como el ser iterativo, sin embargo podemos decir que no es el más certero a la hora de dar soluciones, ya que en algunos casos no las puede dar. De hecho si repetimos los test, aproximadamente de n iteraciones, tenemos que acerta en uno de diez, es decir el $\frac{n}{10}$ aprox de veces nos da una solución.

In [15]:
for i in range(10,3000,1000):
    print(test_finalGauss(i))

/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/829820542.py:54: RuntimeWarning: overflow encountered in matmul
  xk_copy = D_inverse @ b - D_inverse@E@x_copy
/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_5383/829820542.py:54: RuntimeWarning: invalid value encountered in matmul
  xk_copy = D_inverse @ b - D_inverse@E@x_copy


Nuestro programa funciona bien con 10 iteraciones y un total de 0 éxitos.
Nuestro programa funciona bien con 1010 iteraciones y un total de 140 éxitos.
Nuestro programa funciona bien con 2010 iteraciones y un total de 291 éxitos.
